In [37]:
import pandas as pd
import numpy as np

In [38]:
df = pd.read_pickle('files/cleaned.pkl')


In [39]:
# Remove this later
binary_cols = ['Double glazing','Armored door','Elevator','Basement',
             'Intercom','Planning permission obtained']


In [40]:
df.head()

,id,price.mainValue,property.bedroomCount,property.location.box,property.location.latitude,property.location.locality,property.location.longitude,property.location.number,property.location.postalCode,property.location.street,...,As built plan,Bedroom 1 surface,Bedroom 2 surface,Covered parking spaces,Intercom,Terrace surface,Basement,Floor :,Number of floors,Elevator
1,8745713.0,185000.0,2.0,b61,51.185615,Berchem,4.441640,118,2600,Fruithoflaan,...,No,NaN,NaN,1.0,NaN,10.0,1.0,6.0,NaN,1.0
2,8745715.0,280000.0,3.0,b61,51.185615,Berchem,4.441640,118,2600,Fruithoflaan,...,No,NaN,NaN,1.0,NaN,35.0,1.0,6.0,NaN,1.0
4,8744282.0,184000.0,1.0,1R,51.113038,Niel,4.327348,64,2845,Edward Claessenslaan,...,No,NaN,NaN,1.0,NaN,NaN,NaN,1.0,2.0,NaN
5,8664757.0,149000.0,1.0,NaN,51.221663,Antwerp,4.398299,"1, 217",2000,Kaasstraat,...,No,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,1.0
9,8614460.0,175000.0,2.0,NaN,51.305726,Oostmalle,4.737027,74,2390,Hoogstraatsebaan,...,NaN,NaN,NaN,NaN,NaN,5.0,NaN,1.0,NaN,NaN


In [41]:
df[binary_cols] = df[binary_cols].replace([True,False],[1,0])

In [42]:

nominal_cols = ['property.subtype', 'Flood zone type', 'Heating type','Kitchen type',
            'Building condition', 'Type of building','As built plan','property.location.postalCode'
            ]

df = pd.get_dummies(data= df, columns=nominal_cols)

In [43]:
# Ordinal encoding
ordinal_cols= ['Energy class']
scale_mapper = {'A':1,
                'B':2,
                'C':3,
                'D':4,
                'E':5,
                'F':6,
                'Not specified': np.nan
                }

df[ordinal_cols] = df[ordinal_cols].replace(scale_mapper)

In [44]:
# df[geo] = df[geo].replace([True,False],[1,0])



In [45]:
# Making a dict of all cols with missing values

missing = {col: df[col].isnull().sum() for col in df.columns if df[col].isnull().sum()  != 0}

df[binary_cols] = df[binary_cols].fillna(0)


from sklearn.impute import SimpleImputer
missing_ord= [x for x in missing.keys() if x in ordinal_cols]
imp = SimpleImputer(strategy='median')
df[missing_ord]= imp.fit_transform(df[missing_ord])

missing_num = [x for x in missing.keys() if df[x].isnull().sum() != 0]

from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=2)
df[missing_num] = imputer.fit_transform(df[missing_num])

ValueError: could not convert string to float: ' b61'

In [ ]:
for x in df.columns:
    if df[x].dtype not in ['float64', 'uint8','int64']:
        print(x, df[x].dtype)


In [ ]:
correlated_features = set()
correlation_matrix = df.drop('price.mainValue', axis=1).corr()

for i in range(len(correlation_matrix.columns)):
    for j in range(i):
        if abs(correlation_matrix.iloc[i, j]) > 0.8:
            colname = correlation_matrix.columns[i]
            correlated_features.add(colname)

In [ ]:
df.to_pickle('files/preprocessed.pkl')
df.to_csv('files/preprocessed.csv')

In [ ]:
df['glaziery']

In [ ]:
df[binary_cols]